# Real-time face detection & ID identification model for fast check-in.

# Install necessary packages

In [1]:
%%shell
pip install facenet-pytorch
pip install mmcv

# Import Dependencies

In [2]:
from IPython.display import Javascript, Image
from IPython import display as dis
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import numpy as np
import io
import html
import time
from IPython.core.display import Video
from facenet_pytorch import MTCNN
import torch
import mmcv, cv2
import PIL
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from google.colab import files
from google.colab import drive
import os
from os import listdir
from os.path import isfile, join
import re
import shutil
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
import torch.optim as optim
from tqdm import tqdm
from skimage.morphology import disk
from skimage.filters.rank import autolevel
from torch.autograd import Variable

use_cuda = True

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
class FastMTCNN(object):
    """Fast MTCNN implementation."""
    
    def __init__(self, stride, resize=1, *args, **kwargs):
        """Constructor for FastMTCNN class.
        
        Arguments:
            stride (int): The detection stride. Faces will be detected every `stride` frames
                and remembered for `stride-1` frames.
        
        Keyword arguments:
            resize (float): Fractional frame scaling. [default: {1}]
            *args: Arguments to pass to the MTCNN constructor. See help(MTCNN).
            **kwargs: Keyword arguments to pass to the MTCNN constructor. See help(MTCNN).
        """
        self.stride = stride
        self.resize = resize
        self.mtcnn = MTCNN(*args, **kwargs)
        
    def __call__(self, frames):
        """Detect faces in frames using strided MTCNN."""
        if self.resize != 1:
            frames = [
                cv2.resize(f, (int(f.shape[1] * self.resize), int(f.shape[0] * self.resize)))
                    for f in frames
            ]
                      
        boxes, probs = self.mtcnn.detect(frames[::self.stride])

        faces = []
        for i, frame in enumerate(frames):
            box_ind = int(i / self.stride)
            if boxes[box_ind] is None:
                continue
            for box in boxes[box_ind]:
                box = [int(b) for b in box]
                faces.append(frame[box[1]:box[3], box[0]:box[2]])
        
        return faces

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))
# currently using, for face detection
mtcnn = MTCNN(keep_all=True, min_face_size=224, device=device)
# A faster model, for future performance improvment 
mtcnn_fast = FastMTCNN(keep_all=True, min_face_size=224, device=device,stride=4)

Running on device: cuda:0


# Part A: Extracting training data from input videos.

In [ ]:
uploaded_video = files.upload() # For uploading training video to colab, click cancel if have no file to upload
user_name = "ROY"          #User name of the above uploaded video

## Frames extraction: Extract each frame of user face, save to google drive

In [ ]:
video = mmcv.VideoReader('/content/roy_glasses.mov') #Select training video here
frames = [Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) for frame in video] #Extracting frames from video
frames_tracked = []
path = '/content/drive/MyDrive/ut/MIE1517_group_project/ImageDataset/ROY'               #Put in the output directory you want 
os.mkdir(path)
os.chdir(path)
for i, frame in enumerate(frames):
    print('\rTracking frame: {}'.format(i + 1), end='')
    img = np.array(frame)    
    faces, _ = mtcnn.detect(frame)                                           #Return coordinates of faces from mtcnn model

    if i%5==0:

      if faces is not None: 
        for (x,y,w,h) in faces:
          x_w_diff = int(w-x)
          y_h_diff = int(h-y)
          if x_w_diff > 223. and y_h_diff >223.:
            x_w_mid = int(x+(w-x)/2)
            y_h_mid = int(y+(h-y)/2)
            selected_x = x_w_mid - 155
            selected_w = x_w_mid + 155
            selected_y = y_h_mid - 260
            selected_h = y_h_mid + 50           
            selected_img = img[selected_y:selected_h, selected_x:selected_w, :] #Adjustment for cropping the correct face images.

            faces = Image.fromarray(selected_img).resize((224,224))             #Resizing the image to desired input size.
      
            gray_img = cv2.cvtColor(np.array(faces), cv2.COLOR_RGB2GRAY)        #Converting to Gray Scale images for training
            cv2.imwrite(user_name + str(i) + '.png', gray_img)
            #plt.imshow(gray_img, cmap = 'gray')                                #Displaying images only for developing, commented out in actual practice.
            #plt.show() 
    else:
      continue                                                     
          

print('\nDone')

Tracking frame: 86
Done


## ID identification MODEL and TRAINING method CAN be placed below.

In [11]:
def split_data(dataset_path, batch_size=64):

    transform = transforms.Compose([transforms.Resize((224, 224)),
                    transforms.RandomHorizontalFlip(),  # flip images
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),])

    training_dataset = datasets.ImageFolder(root = dataset_path, transform=transform)

    print(training_dataset.class_to_idx)

    indices = list(range(len(training_dataset)))

    np.random.shuffle(indices)

    split_train_val = int(0.8 * len(indices))

    train_indices, val_indices = indices[:split_train_val], indices[split_train_val:]

    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader, val_loader = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, sampler=train_sampler), \
                  torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, sampler=val_sampler)
                    
    return train_loader, val_loader

In [13]:
class AlexNetModel(nn.Module):

    def __init__(self, outputs):
        super(AlexNetModel, self).__init__()
        self.name = 'AlexNetModel'
        self.conv1 = nn.Conv2d(256, 30, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(120, 64)
        self.fc2 = nn.Linear(64, outputs*2)
        self.outputs = outputs

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 120)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        mean, variance = torch.split(x, self.outputs, dim=1)
        variance = F.softplus(variance) + 1e-6
        return mean, variance


In [14]:
class GaussianMixtureAlex(nn.Module):
    """ Gaussian mixture MLP which outputs are mean and variance.

    Attributes:
        models (int): number of models
        inputs (int): number of inputs
        outputs (int): number of outputs
        hidden_layers (list of ints): hidden layer sizes

    """
    def __init__(self, num_models=5, outputs=1):
        super(GaussianMixtureAlex, self).__init__()
        self.name = 'GaussianMixtureAlex'
        self.num_models = num_models
        self.outputs = outputs
        for i in range(self.num_models):
            model = AlexNetModel(outputs=self.outputs)
            setattr(self, 'model_'+str(i), model)
            
    def forward(self, x):
        # connect layers
        means = []
        variances = []
        for i in range(self.num_models):
            model = getattr(self, 'model_' + str(i))
            mean, var = model(x)
            means.append(mean)
            variances.append(var)
        means = torch.stack(means)
        mean = means.mean(dim=0)
        variances = torch.stack(variances)
        variance = (variances + means.pow(2)).mean(dim=0) - mean.pow(2)
        return mean, variance 

In [15]:
# function to save checkpoint

def OneHotEncoder(ls, class_num = 11):
  unique_ls = list(set(ls))
  onehot_encoded = np.zeros((len(ls), class_num))
  for i in range(len(ls)):
    onehot_encoded[i][ls[i]] = 1
  return onehot_encoded

def save_checkpoint(model, batch_size, lr, epoch):
  model_path = "{}_{}_{}_{}".format(model.name, batch_size, lr, epoch)
  torch.save(model, model_path)
  print('Checkpoint of {} has been stored successfully!',format(model_path))

def NLLloss(y, mean, var):
    """ Negative log-likelihood loss function. """
    return (torch.log(var) + ((y - mean).pow(2))/var).sum()

def get_accuracy(model, data_loader, grey_images_flag = False):
  correct = 0
  total = 0

  alexnet = torchvision.models.alexnet(pretrained=True)

  for imgs, labels in data_loader:
    #############################################
    #To Enable GPU Usage

    if grey_images_flag:

      grey_images = torchvision.transforms.Grayscale()(imgs)

      imgs = torch.tensor(np.tile(grey_images, [1,3,1,1]))

    imgs = alexnet.features(imgs)

    if use_cuda and torch.cuda.is_available():
      imgs = imgs.cuda()
      labels = labels.cuda()
    #############################################
    
    mean, _ = model(imgs)
    
    #select index with maximum prediction score
    pred = mean.max(1, keepdim=True)[1]
    correct += pred.eq(labels.view_as(pred)).sum().item()
    total += imgs.shape[0]
  return correct / total

def predict(model, data_loader, grey_images_flag = False):

  alexnet = torchvision.models.alexnet(pretrained=True)

  pred_ls, var_ls, std_ls = [], [], []

  for imgs, _ in data_loader:
    #############################################
    #To Enable GPU Usage

    if grey_images_flag:

      grey_images = torchvision.transforms.Grayscale()(imgs)

      imgs = torch.tensor(np.tile(grey_images, [1,3,1,1]))

    imgs = alexnet.features(imgs)

    if use_cuda and torch.cuda.is_available():
      imgs = imgs.cuda()
    #############################################
    
    mean, var = model(imgs)
    
    #select index with maximum prediction score
    pred = torch.argmax(mean, dim=1)

    pred = pred.cpu().detach().numpy()

    var = var.cpu().detach().numpy()

    variance = [var[i][val] for i, val in enumerate(pred)]

    std = np.sqrt(variance)

    pred_ls = [*pred_ls, *pred]

    var_ls = [*var_ls, *variance]

    std_ls = [*std_ls, *std]

  return pred_ls, var_ls, std_ls


def train(model, dataset_path, opt, batch_size = 64, learning_rate=0.01, epochs=30, grey_images_flag = False):

  #############################################
  # input: grey_images_flag: boolean if gray image conversion is needed
  #############################################

  train_loader, val_loader = split_data(dataset_path, batch_size = batch_size)

  criterion = NLLloss

  # criterion = nn.CrossEntropyLoss()

  optimizer = opt(model.parameters(), lr=learning_rate)
    
  iters, losses, train_acc, val_acc = [], [], [], []
  
  alexnet = torchvision.models.alexnet(pretrained=True)

  n = 0 # the number of iterations
  for epoch in range(epochs):
    for imgs, labels in tqdm(iter(train_loader)):

      if grey_images_flag:

        grey_images = torchvision.transforms.Grayscale()(imgs)

        imgs = torch.tensor(np.tile(grey_images, [1,3,1,1]))

      imgs = alexnet.features(imgs)

      #############################################
      #To Enable GPU Usage
      if use_cuda and torch.cuda.is_available():
        imgs = imgs.cuda()

        one_hot_labels = torch.from_numpy(OneHotEncoder(labels))

        labels = one_hot_labels.cuda()
      #############################################
  
      mean, var = model(imgs) 
      # print(mean.shape)
      # print(var.shape)
      # print(labels.shape)

      # output = model(imgs)

      loss = NLLloss(labels, mean, var)

      # loss = criterion(output, labels)

      # loss = criterion(m(mean), labels)

      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

    # save the current training information
    iters.append(n)
    losses.append(float(loss)/batch_size)
    train_acc.append(get_accuracy(model, train_loader, grey_images_flag = grey_images_flag))
    val_acc.append(get_accuracy(model, val_loader, grey_images_flag = grey_images_flag))
    n += 1
      
    if ((epoch+1) % 25 == 0):
      save_checkpoint(model, batch_size, learning_rate, epoch+1)

  # plotting
  plt.title("Training Curve")
  plt.plot(iters, losses, label="Train")
  plt.xlabel("Iterations")
  plt.ylabel("Loss")
  plt.show()

  plt.title("Training Curve")
  plt.plot(iters, train_acc, label="Train")
  plt.plot(iters, val_acc, label="Validation")
  plt.xlabel("Iterations")
  plt.ylabel("Training Accuracy")
  plt.legend(loc='best')
  plt.show()

  print("Final Training Accuracy: {}".format(train_acc[-1]))
  print("Final Validation Accuracy: {}".format(val_acc[-1]))

In [16]:
class_num = 11

model = GaussianMixtureAlex(outputs = class_num)

data_path = '/content/drive/MyDrive/ut/MIE1517_group_project/ImageDataset/'

if use_cuda and torch.cuda.is_available():
  model.cuda()
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')
  
train(model, data_path, opt = optim.Adam, batch_size = 32, learning_rate=0.001, epochs=50, grey_images_flag = True)

CUDA is available!  Training on GPU ...
{'0_BOB': 0, '10_ZC': 1, '1_CZJ': 2, '2_DYY': 3, '3_HanHan': 4, '4_JCL': 5, '5_ML': 6, '6_QRH': 7, '7_ROY': 8, '8_SL': 9, '9_WPL': 10}


 38%|███▊      | 3/8 [00:04<00:07,  1.51s/it]


KeyboardInterrupt: ignored

In [17]:
train_loader, val_loader = split_data(data_path, batch_size = 32)

{'0_BOB': 0, '10_ZC': 1, '1_CZJ': 2, '2_DYY': 3, '3_HanHan': 4, '4_JCL': 5, '5_ML': 6, '6_QRH': 7, '7_ROY': 8, '8_SL': 9, '9_WPL': 10}


In [18]:
pred, variance, std = predict(model, val_loader, grey_images_flag = True)

# Part B: Real time face detection & ID identification 

## Helper method for creating video streaming

In [19]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply): 
  image_bytes = b64decode(js_reply.split(',')[1])        # decode base64 image
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8) # convert bytes to numpy array
  img = cv2.imdecode(jpg_as_np, flags=1)                 # decode numpy array into OpenCV BGR image

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(ipt):
  # ipt: Numpy array (pixels) containing rectangle to overlay on video stream.
  # bytes: Base64 image byte string
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(ipt, 'RGBA')
  iobuf = io.BytesIO() 
  bbox_PIL.save(iobuf, format='png')                                                          # format bbox into png for return
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8'))) # format return string

  return bbox_bytes
  
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;   
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 720, 720);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = '<span style="color: red; font-weight: bold;">' + 'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 720; //video width; 1280
      captureCanvas.height = 720; //video height; 720
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

## Face detection model implementations

### Re-implementing face detection model, in case some unexpected changes were made in part A

In [20]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))
# currently using, for face detection
mtcnn = MTCNN(keep_all=True, min_face_size=224, device=device)
# A faster model
mtcnn_fast = FastMTCNN(keep_all=True, min_face_size=224, device=device,stride=4)

Running on device: cuda:0


### Turning on webcam, start real-time ID recognition

In [21]:
model = torch.load("/content/GaussianMixtureAlex_32_0.001_25")

In [43]:
from torch.functional import Tensor
video_stream()
label_html = 'Capturing Video...'
bbox = ''
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break
    
    img = js_to_image(js_reply["img"])                            #Calling helper method to convert JS response to OpenCV Image

    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #frame_fast = torch.from_numpy(rgb_img) 
    #frame_fast = torch.stack([frame_fast], dim=0)                #Prepared input frame for mtcnn_fast face detection model
    frame = Image.fromarray(rgb_img)                              #Prepared input frame for mtcnn face detection model

    faces, _ = mtcnn.detect(frame)                                #Return coordinates of faces from mtcnn model
    #faces_fast = mtcnn_fast(frame_fast)                          #Higher throughput model, return captured face images, not coordinates.
    #plt.imshow(faces_fast[0].detach().numpy(), cmap = 'gray')    #Displaying images only for developing, commented out in actual practice.
    #plt.show()

    bbox_array = np.zeros([720,720,4], dtype=np.uint8)            #Creating a transparent overlay for drawing bounding box

    if faces is not None: 
      for (x,y,w,h) in faces:
        x_w_diff = int(w-x)
        y_h_diff = int(h-y)
        if x_w_diff > 223. and y_h_diff >223.:                    #Size filtering, detected faces need to be clear enough (224 x 224)
          x_w_mid = int(x+(w-x)/2)                                #Finding the middle point along the x-axis
          y_h_mid = int(y+(h-y)/2)                                #Finding the middle point along the y-axis
          selected_x = x_w_mid - 140
          selected_w = x_w_mid + 140
          selected_y = y_h_mid - 260
          selected_h = y_h_mid + 20                               
 
          selected_img = img[selected_y:selected_h, selected_x:selected_w, :]  #Adjustment for cropping the correct face images.

          try:
            faces = Image.fromarray(selected_img).resize((224,224))
          except:
            label_html = "Move face to the center"
            continue
    
          gray_img = cv2.cvtColor(np.array(faces), cv2.COLOR_RGB2GRAY)
          #plt.imshow(gray_img, cmap = 'gray')                                 #Displaying face images during developing, commented out in actual practice
          #plt.show()
          
          input_for_face_recognition = torch.from_numpy(gray_img) 
          input_for_face_recognition = torch.stack([input_for_face_recognition], dim=0) #Prepared input in tensor format, Ready for our ID detection model
          

          # Facial identification process will be placed here

          transform = transforms.Compose([transforms.Resize((224, 224)),
                transforms.RandomHorizontalFlip(),  # flip images
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),])

          faces = transform(faces)

          alexnet = torchvision.models.alexnet(pretrained=True)

          grey_images = torchvision.transforms.Grayscale()(faces)

          imgs = torch.tensor(np.tile(grey_images, [1,3,1,1]))
          
          imgs = alexnet.features(imgs)
          if use_cuda and torch.cuda.is_available():
            imgs = imgs.cuda()
          #############################################

          mean, var = model(imgs)

          confidence = nn.Softmax(dim=1)(mean)

          confidence = torch.max(confidence)

          #select index with maximum prediction score
          pred = torch.argmax(mean, dim=1)
          pred = pred.cpu().detach().numpy()
          result = str(pred[0]) + " Confidence {}  Uncertainty {}".format(str(confidence.cpu().detach()) ,var[0][pred[0]])
          # print(result)
          label_html = result

          box_height = (w,int(h-(20)))                                           #Adjusting the size of the bounding box to indicate detected faces.
          bbox_array = cv2.rectangle(bbox_array,(x,y), box_height, (0,255,0), 2) #Bounding box size and colour
      bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255  
      bbox_bytes = bbox_to_bytes(bbox_array)                                     # converting overlay of bbox into bytes  
      bbox =bbox_bytes                                                           # update bbox for the next frame  

<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored

In [ ]:
%%shell
jupyter nbconvert --to html Project_Face_detection.ipynb